In [ ]:
import random as rd
import requests as rq
import time
import json


In [ ]:
def pingHTTP(ping):
    print(ping)
    print(requests.post('http://0.0.0.0:8000/run_popen', data=json.dumps(ping)).json())
    
def create_route_list(number_hosts, number_request):
    routes = [] 
    pings = []
    
    for i in range(number_request):
        src_host = rd.randint(1, number_hosts)
        dst_host = rd.randint(1, number_hosts)
        while src_host == dst_host:
            dst_host = rd.randint(1, number_hosts)
            
        route = {
            "src_host": src_host,
            "dst_host": dst_host
            }
        routes.append(route)
        cmd = (
            f"""curl -X 'POST' 'http://0.0.0.0:8001/upload_speed/' -H 'accept: application/json' -H 'Content-Type: application/json' -d '{{"ip": "10.0.0.{dst_host}", "port": 8001, "size": 50}}'"""
        )
        pings.append(
            {
                "hostname": "h" + str(src_host),
                "cmd": cmd,
                "wait": True
            }
        )
    return routes, pings

def init_host_list(hosts):
    initHosts_list = []
    
    for i_host in hosts:
        # hosts.add("h"+str(route["src_host"]))
        # hosts.add("h"+str(route["dst_host"]))
        initHosts_list.append({
            "hostname": "h"+str(i_host),
            "cmd": '''
                    ../../venv11/bin/python3.11 \
                    mn_app/simplehttpserver/http_file_transfer.py 0.0.0.0 8001
                    ''',
            "wait": False
        })
    return initHosts_list

In [ ]:
class RandomRequestGen:
    def __init__(self, *args, **kwargs) -> None:
        self.server_ls = kwargs['server_ls']
        self.client_ls = kwargs['client_ls']
        self.num_single_rq = kwargs['num_single_rq']
        self.routes = [] 
    

    def get_route_single(self):
        for i in range(self.num_single_rq):
            choice_client = rd.choice(self.client_ls)
            choice_server = rd.choice(self.server_ls)
            
            route = {
                'src_host': choice_client,
                'dst_host': choice_server,
            }
            self.routes.append()
        
    def get_route_multi(self):
        for i in range(self.num_single_rq):
            choice_client = rd.choice(self.client_ls)
            choice_server = rd.choice(self.server_ls)
            
            route = {
                'src_host': choice_client,
                'dst_host': choice_server,
            }
            self.routes.append()
    
    # Don't have api to implement here
    def send_route_request(self):
        for i in range(self.num_single_rq):
            rq.post('', data=json.dumps())



In [ ]:
class PingTraffic:
    def __init__(self, *args, **kwargs) -> None:
        self.timeout = kwargs['timeout']
        self.count = kwargs['count']

    def ping_single(self, src_host: str, dst_host: str):
        request = {
            "src_hostname": src_host,
            "dst_hostname": dst_host,
            "timeout": '3',
            "count": '3'
        }
        return rq.post('http://0.0.0.0:8000/ping_single', json.dumps(request)).json()
        
    
    def ping_multi(self, host_ls: str):
        '''
           Ping between all host in list
           ex input: ['h1', 'h2', 'h3',...] 
        '''
        request = {
            "hostname_list": host_ls,
            "timeout": '3'
        }
        return rq.post('http://0.0.0.0:8000/ping', json.dumps(request)).json()
    
    def ping_all(self):
        return rq.post('http://0.0.0.0:8000/pingall', json.dumps({"timeout": "3"})).json()

# PingTraffic(timeout=3, count=3).ping_single('h2','h3')
# PingTraffic(timeout=3, count=3).ping_multi(['h2','h3'])
PingTraffic(timeout=3, count=3).ping_all()


In [46]:
req = {
    "hostname": f"h1",
    "cmd": '''
            ../../venv11/bin/python3.11 \
            mn_app/simplehttpserver/http_file_transfer.py 0.0.0.0 8001
            ''',
    "wait": False
}
req ={
    "hostname": "h1",
    "cmd": '''
            ../../venv11/bin/python3.11 \
            mn_app/simplehttpserver/http_file_transfer.py 0.0.0.0 8001
            ''',
}
rq.post('http://0.0.0.0:8000/run_xterm', data=json.dumps(req))

<Response [200]>

In [56]:
req = {
    "hostname": f"h1",
    "cmd": '''
        ../../venv11/bin/python3.11 \
        mn_app/simplehttpserver/http_file_transfer.py 0.0.0.0 8001
        ''',
    "wait": False
}
rq.post('http://0.0.0.0:8000/run_popen', data=json.dumps(req)).text

'{"result":true}'

In [58]:
# check status
req = {
    "hostname": "h1",
    "cmd": "curl -X 'GET' 'http://0.0.0.0:8001/'",
    "wait": True
}
result = rq.post('http://0.0.0.0:8000/run_popen', data=json.dumps(req)).json()

json.loads(result['result'])['status']

True

In [60]:
host_ls = rq.get('http://0.0.0.0:8000/host').json()

class HttpFileTransferController:
    def __init__(self, 
                 
                 host_ls: dict, app_port: int):
        self.app_port = app_port
        self.host_ls = host_ls
        
        for hostname, host_addr in host_ls.items():
            
            if self.is_online(hostname) is True:
                continue
            
            req = {
                "hostname": f"h1",
                "cmd": f" ../../venv11/bin/python3.11 \
                        mn_app/simplehttpserver/http_file_transfer.py 0.0.0.0 {self.app_port} \
                        ",
                "wait": False
            }
            rq.post('http://0.0.0.0:8000/run_popen', data=json.dumps(req)).json()

    @property
    def is_online(self, hostname):
        req = {
            "hostname": f"{hostname}",
            "cmd": f"curl -X 'GET' 'http://0.0.0.0:{self.app_port}'",
            "wait": True
        }
        result = rq.post('http://0.0.0.0:8000/run_popen', data=json.dumps(req)).json()
        return json.loads(result['result'])['status']
    
    def file_upload(self, src_hostname: str, dst_hostname, size: int | None = 10):
        req = ({
            "hostname": f"{src_hostname}",
            "cmd":f"""curl -X 'POST' 'http://0.0.0.0:{self.app_port}/upload_speed/' \
                 -H 'accept: application/json' -H \
                 'Content-Type: application/json' -d \
                 '{{"ip": "{self.host_ls[dst_hostname]["ip"]}", \
                 "port": {self.app_port}, "size": 50}}'""",
            "wait": False
        })
    
    def file_download(self, src_hostname: str, dst_hostname, size: int | None = 10):
        req = ({
            "hostname": f"{src_hostname}",
            "cmd":f"""curl -X 'POST' 'http://0.0.0.0:{self.app_port}/download_speed/' \
                 -H 'accept: application/json' -H \
                 'Content-Type: application/json' -d \
                 '{{"ip": "{self.host_ls[dst_hostname]["ip"]}", \
                 "port": {self.app_port}, "size": 50}}'""",
            "wait": False
        })
        result = rq.post('http://0.0.0.0:8000/run_popen', data=json.dumps(req)).json()
        return json.loads(result['result'])['status']
    
    def respone_time(self, src_hostname, dst_hostname):
        req = ({
            "hostname": f"{src_hostname}",
            "cmd":f"""curl -X 'POST' 'http://0.0.0.0:{self.app_port}/download_speed/' \
                 -H 'accept: application/json' -H \
                 'Content-Type: application/json' -d \
                 '{{"ip": "{self.host_ls[dst_hostname]["ip"]}", \
                 "port": {self.app_port}}}'""",
            "wait": False
        })
        result = rq.post('http://0.0.0.0:8000/run_popen', data=json.dumps(req)).json()
        return json.loads(result['result'])['status']